##### Ollama C++ Code Generation Test

This notebook tests Ollama's ability to generate working C++ code from Python with proper error handling and debugging.

In [25]:
# Basic imports
import os
import subprocess
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv(override=True)

True

In [26]:
# Setup Ollama connection
ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')
OLLAMA_MODEL = "llama3.2"

In [27]:
system_message_test = "You are a C++ code generator for Windows. "
system_message_test += "CRITICAL RULES: "
system_message_test += "1. Output ONLY valid C++ code - NO explanations, NO notes, NO text outside code "
system_message_test += "2. Start immediately with #include statements "
system_message_test += "3. Never write 'Here is the code' or 'Note that' or any explanatory text "
system_message_test += "4. ALWAYS include: #include <iostream>, #include <chrono>, #include <cstdint>, #include <vector> "
system_message_test += "5. Use int64_t for negative numbers like -10 "
system_message_test += "6. Use uint64_t for large positive numbers like 4294967296ULL "
system_message_test += "7. NO Python keywords like 'yield' - use simple classes with next() methods "
system_message_test += "8. Match variable names EXACTLY from function parameters "
system_message_test += "9. NEVER increment const references "
system_message_test += "10. Respond with C++ code ONLY - no markdown, no explanations, just code"

In [28]:
def test_ollama_simple(python_code):
    """Test Ollama with a simple Python example"""
    user_prompt = f"Convert this Python to C++:\n\n{python_code}"
    
    messages = [
        {"role": "system", "content": system_message_test},
        {"role": "user", "content": user_prompt}
    ]
    
    response = ollama_via_openai.chat.completions.create(
        model=OLLAMA_MODEL,
        messages=messages,
        stream=False
    )
    
    return response.choices[0].message.content

In [29]:
def clean_and_save_cpp(cpp_code, filename="test.cpp"):
    """Clean C++ code and save to file"""
    # Remove markdown formatting
    code = cpp_code.replace("```cpp", "").replace("```", "")
    
    # More aggressive cleaning based on #codebase patterns
    lines = code.split('\n')
    cleaned_lines = []
    
    in_code_block = False
    
    for line in lines:
        stripped = line.strip()
        
        # Skip explanatory lines completely
        if (stripped.startswith('Here ') or 
            stripped.startswith('Note') or
            stripped.startswith('This ') or
            stripped.startswith('The ') or
            stripped.startswith('I ') or
            stripped.startswith('Here\'s') or
            '`' in stripped or
            stripped == '' and not in_code_block):
            continue
            
        # Look for actual C++ code patterns
        if (stripped.startswith('#include') or
            stripped.startswith('int ') or
            stripped.startswith('void ') or
            stripped.startswith('class ') or
            stripped.startswith('struct ') or
            stripped.startswith('using ') or
            stripped.startswith('std::') or
            '{' in stripped or
            '}' in stripped):
            in_code_block = True
        
        if in_code_block:
            cleaned_lines.append(line)
    
    cleaned_code = '\n'.join(cleaned_lines)
    
    with open(filename, 'w') as f:
        f.write(cleaned_code)
    
    return cleaned_code

In [30]:
def compile_and_run_cpp(filename="test.cpp"):
    """Compile and run C++ code with detailed error reporting"""
    exe_name = filename.replace('.cpp', '.exe')
    
    try:
        # Compile
        compile_cmd = ["g++", "-O3", "-std=c++17", "-o", exe_name, filename]
        compile_result = subprocess.run(compile_cmd, capture_output=True, text=True)
        
        if compile_result.returncode != 0:
            return f"COMPILATION ERROR:\n{compile_result.stderr}"
        
        # Run
        run_result = subprocess.run([exe_name], capture_output=True, text=True)
        
        if run_result.returncode != 0:
            return f"RUNTIME ERROR:\n{run_result.stderr}"
        
        return f"SUCCESS:\n{run_result.stdout}"
        
    except Exception as e:
        return f"EXCEPTION: {str(e)}"

In [31]:
# Test 1: Simple Python example (based on #codebase patterns)
simple_python = """
import time

def calculate_sum(n):
    total = 0
    for i in range(1, n+1):
        total += i
    return total

start_time = time.time()
result = calculate_sum(1000000)
end_time = time.time()

print(f"Result: {result}")
print(f"Time: {end_time - start_time:.6f} seconds")
"""

print("Testing simple Python example...")
cpp_code = test_ollama_simple(simple_python)
print("Generated C++ code:")
print(cpp_code)

Testing simple Python example...
Generated C++ code:
```cpp
#include <iostream>
#include <chrono>

class SumCalculator {
public:
    virtual size_t total() const = 0;
};

struct NaiveSumCalculator : public SumCalculator {
    size_t calculate(size_t n) override {
        size_t total = 0;
        for (size_t i = 1; i <= n; ++i)
            total += i;

        return total;
    }

private:
    size_t m_n;
};

class OptimizedSumCalculator : public SumCalculator {
public:
    OptimizedSumCalculator(size_t n) : m_n(n) {}

    size_t total() const override {
        const uint64_t sum = (uint64_t)(n + 1) * ((uint64_t)n / 2);
        return static_cast<size_t>(sum);
    }

private:
    uint32_t n;
};

int main() {
#include <cstdint>

    OptimizedSumCalculator calculator(1000000);

    auto start_time = std::chrono::high_resolution_clock::now();
    size_t result = calculator.total();
    auto end_time = std::chrono::high_resolution_clock::now();

    auto duration = std::chrono::duration_c

In [32]:
# Clean and save the generated code
cleaned_code = clean_and_save_cpp(cpp_code, "test_simple.cpp")
print("\nCleaned C++ code:")
print(cleaned_code)


Cleaned C++ code:
#include <iostream>
#include <chrono>

class SumCalculator {
public:
    virtual size_t total() const = 0;
};

struct NaiveSumCalculator : public SumCalculator {
    size_t calculate(size_t n) override {
        size_t total = 0;
        for (size_t i = 1; i <= n; ++i)
            total += i;

        return total;
    }

private:
    size_t m_n;
};

class OptimizedSumCalculator : public SumCalculator {
public:
    OptimizedSumCalculator(size_t n) : m_n(n) {}

    size_t total() const override {
        const uint64_t sum = (uint64_t)(n + 1) * ((uint64_t)n / 2);
        return static_cast<size_t>(sum);
    }

private:
    uint32_t n;
};

int main() {
#include <cstdint>

    OptimizedSumCalculator calculator(1000000);

    auto start_time = std::chrono::high_resolution_clock::now();
    size_t result = calculator.total();
    auto end_time = std::chrono::high_resolution_clock::now();

    auto duration = std::chrono::duration_cast<std::chrono::milliseconds>(end_time -

In [33]:
# Try to compile and run
result = compile_and_run_cpp("test_simple.cpp")
print("\nCompilation and execution result:")
print(result)


Compilation and execution result:
COMPILATION ERROR:
test_simple.cpp:10:12: error: 'size_t NaiveSumCalculator::calculate(size_t)' marked 'override', but does not override
   10 |     size_t calculate(size_t n) override {
      |            ^~~~~~~~~
test_simple.cpp: In constructor 'OptimizedSumCalculator::OptimizedSumCalculator(size_t)':
test_simple.cpp:24:40: error: class 'OptimizedSumCalculator' does not have any field named 'm_n'
   24 |     OptimizedSumCalculator(size_t n) : m_n(n) {}
      |                                        ^~~



In [34]:
# Test 2: The problematic LCG example (simplified)
lcg_python = """
def lcg_simple(seed, count):
    a = 1664525
    c = 1013904223
    m = 4294967296  # 2^32
    
    value = seed
    results = []
    
    for i in range(count):
        value = (a * value + c) % m
        results.append(value)
    
    return results

# Test it
numbers = lcg_simple(42, 5)
print("Generated numbers:", numbers)
"""

print("Testing LCG example...")
cpp_code2 = test_ollama_simple(lcg_python)
print("Generated C++ code:")
print(cpp_code2)

Testing LCG example...
Generated C++ code:
```cpp
#include <iostream>
#include <chrono>
#include <cstdint>
#include <vector>

uint64_t const a = 1664525;
uint64_t const c = 1013904223;
uint64_t const m = 4294967296ULL;

std::vector<uint64_t> lcg_simple(uint64_t seed, uint64_t count) {
    std::vector<uint64_t> results;
    uint64_t value = seed;

    for (uint64_t i = 0; i < count; ++i) {
        value = (a * value + c) % m;
        results.push_back(value);
    }

    return results;
}

int main() {
    auto numbers = lcg_simple(42, 5);
    for (const auto& num : numbers) {
        std::cout << num << " ";
    }
    std::cout << std::endl;

    return 0;
}
```


In [35]:
# Clean and test the LCG code
cleaned_code2 = clean_and_save_cpp(cpp_code2, "test_lcg.cpp")
result2 = compile_and_run_cpp("test_lcg.cpp")
print("\nLCG compilation and execution result:")
print(result2)


LCG compilation and execution result:
SUCCESS:
1083814273 378494188 2479403867 955863294 1613448261 



In [36]:
# If LCG works, try the max subarray (very simplified)
if "SUCCESS" in result2:
    simple_max_subarray = """
def max_subarray_simple(arr):
    max_sum = float('-inf')
    
    for i in range(len(arr)):
        current_sum = 0
        for j in range(i, len(arr)):
            current_sum += arr[j]
            if current_sum > max_sum:
                max_sum = current_sum
    
    return max_sum

# Test with simple array
test_array = [-2, 1, -3, 4, -1, 2, 1, -5, 4]
result = max_subarray_simple(test_array)
print(f"Max subarray sum: {result}")
"""
    
    print("Testing simplified max subarray...")
    cpp_code3 = test_ollama_simple(simple_max_subarray)
    cleaned_code3 = clean_and_save_cpp(cpp_code3, "test_max_subarray.cpp")
    result3 = compile_and_run_cpp("test_max_subarray.cpp")
    print("\nMax subarray result:")
    print(result3)
else:
    print("Skipping max subarray test due to LCG failure")

Testing simplified max subarray...

Max subarray result:
SUCCESS:
Max subarray sum: 6

